# Introductions

Using tools from the [Bosch Small Traffic Lights Dataset](https://hci.iwr.uni-heidelberg.de/node/6132).

# Modules

In [ ]:
import sys
sys.path.append("./bstld")

In [14]:
import os

import bstld
from bstld.dataset_stats import quick_stats 

In [6]:
print(sys.path)

['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/root/.ipython', './bstld']


# Module Constants

In [12]:
BOSCH_DATASET_DIR = "data/bosch_dataset"

# Tests datas

In [15]:
_train_path = os.path.join(BOSCH_DATASET_DIR, "train.yaml")
print(_train_path)

data/bosch_dataset/train.yaml


In [16]:
quick_stats(_train_path)

Number of images: 5093
Number of traffic lights: 10756 

Minimum width: 1.1224768885000458
Average width: 11.175106667325696
median width: 8.55396570789992
maximum width: 98.0 

Minimum height: 0.24943930860001728
Average height: 24.323216404050356
median height: 18.925365134599986
maximum height: 207.0 

Minimum size: 0.2799898589869501
Average size: 404.5161008940251
median size: 158.80036482309796
maximum size: 20286.0 

Labels:
	Yellow: 444
	GreenRight: 13
	RedRight: 5
	GreenStraightRight: 3
	RedStraightLeft: 1
	RedStraight: 9
	occluded: 170
	RedLeft: 1092
	off: 726
	Green: 5207
	GreenStraightLeft: 1
	GreenLeft: 178
	GreenStraight: 20
	Red: 3057
